In [1]:
import sqlite3
import pandas as pd
import numpy as np

#Selecionar arquivo CSV na janela do Windows que servirá como base de dados
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])

In [2]:
#### le arquivo CSV e cria DataFrame "df" ####

df = pd.read_csv(file_path)

# Exclui linhas com valores faltantes
df.dropna(inplace=True)

# Exclui linhas com valores inválidos
condicaoNotasInvalidas = ((df['math score'] > 100) | (df['math score'] < 0) | 
                         (df['reading score'] > 100) | (df['reading score'] < 0) | 
                         (df['writing score'] > 100) | (df['writing score'] < 0))

df.drop(df.loc[condicaoNotasInvalidas].index, inplace=True)

# Mostra dataframe
df.sample(n=5)

In [3]:
### Cria novo DataFrame "dfFato", cópia de "df" mas apenas com os campos de notas ###

dfFato = df[['math score', 'reading score', 'writing score']].copy(deep=True)

#Inserir coluna ID no dataframe fato
dfFato.insert(0, 'ID', dfFato.index)

dfFato.head()

In [4]:
########### Cria tabela fato "tabela_fato" no SQLite ###########

# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Carregar o dataframe para o banco de dados como uma tabela fato
dfFato.to_sql('tabela_fato', conn, if_exists='replace', index=False)

# Fecha a conexão com o banco de dados
conn.close()

In [5]:
########### Alterar tabela_fato para criar chave estrangeira ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

cursor.execute('''
    ALTER TABLE tabela_fato ADD COLUMN ID_tabela_dimensao INTEGER REFERENCES tabela_dimensao(ID)
''')


#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()

In [6]:
########### Cria tabela_dimensao ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Apaga a tabela dimensão se ela já existe
cursor.execute('DROP TABLE IF EXISTS tabela_dimensao')

#Cria a tabela dimensão
cursor.execute('''
    CREATE TABLE tabela_dimensao (
        ID INTEGER PRIMARY KEY,
        gender TEXT,
        race_ethnicity TEXT,
        parental_level_of_education TEXT,
        lunch TEXT,
        test_preparation_course TEXT
    )
''')

# insere os dados na tabela dimensão, usando o índice da tabela fato como chave primária
for i, row in df.iterrows():
    query = f"""
        INSERT INTO tabela_dimensao (id, gender, race_ethnicity, parental_level_of_education, lunch, test_preparation_course)
        VALUES ({i}, "{row['gender']}", "{row['race/ethnicity']}", "{row['parental level of education']}", "{row['lunch']}", "{row['test preparation course']}")
    """
    cursor.execute(query)

#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()


In [7]:
########### Preencher chave estrangeira tabela_fato ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Insere os dados nas colunas de chave estrangeira da tabela_fato
cursor.execute('''
    UPDATE tabela_fato SET ID_tabela_dimensao = ID
''')

#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()

In [8]:
########### Visualizar tabela ###########

# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Executar a consulta SQL para selecionar todos os dados da tabela (Ex.: tabela_dimensao)
consultaTabela = pd.read_sql_query("SELECT * from tabela_dimensao", conn)

# Fechar a conexão com o banco de dados
conn.close()

# Visualizar o dataframe
consultaTabela